In [1]:
import glob
import pandas as pd
import math
from sklearn.neighbors import BallTree
import numpy as np

In [2]:
SRON_paths = glob.glob('C:/Users/hughr/OneDrive/Documents/RMI/SRON/*.csv')
latlons_SRON = []
for p in SRON_paths:
    df = pd.read_csv(p, index_col=0)
    latlons_SRON.append(df)

latlons_SRON = pd.concat(latlons_SRON, axis=0)
latlons_SRON.reset_index(drop=True, inplace=True)
latlons_SRON

,date,time_UTC,lat,lon,source_rate_t/h,uncertainty_t/h,uncertainty
0,20230304.0,18:43:18,32.20,-93.35,12,6.0,NaN
1,20230304.0,20:23:30,31.40,-103.55,12,6.0,NaN
2,20230304.0,20:23:24,31.23,-101.65,13,8.0,NaN
3,20230304.0,20:23:33,31.66,-104.02,23,10.0,NaN
4,20230305.0,08:10:15,17.93,78.45,12,5.0,NaN
...,...,...,...,...,...,...,...
770,20230304.0,08:29:41,19.26,72.87,54,23.0,NaN
771,20230304.0,08:30:31,22.09,72.83,5,4.0,NaN
772,20230304.0,08:32:45,30.17,71.42,20,12.0,NaN
773,20230304.0,08:33:02,31.46,73.07,22,11.0,NaN


In [3]:
latlons_TRACE = pd.read_csv('C:/Users/hughr/OneDrive/Documents/RMI/Climate_TRACE/waste/asset_solid-waste-disposal_emissions.csv')
latlons_TRACE = latlons_TRACE[latlons_TRACE['gas'] == 'ch4'].reset_index(drop=True)
latlons_TRACE

,asset_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,capacity,capacity_units,capacity_factor,activity,activity_units,created_date,modified_date,asset_name,asset_type,st_astext
0,1835960681,KOR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,228071.908800,0.034650,NaN,1.144865e+08,landfill_area_sq_meters,NaN,6.582168e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Sudokwon,Sanitary Landfills,POINT(126.613998413085 37.5772357911111)
1,1835960682,TUR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131856.520200,0.030742,NaN,7.534171e+07,landfill_area_sq_meters,NaN,4.289130e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Odayeri,Sanitary Landfills,POINT(28.8581657 41.2161766)
2,1835960683,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131104.987200,0.034650,NaN,6.413101e+07,landfill_area_sq_meters,NaN,3.783697e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Laogang Landfill,Sanitary Landfill,POINT(121.874575 31.056294)
3,1835960821,USA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,9868.230000,0.006824,NaN,3.108446e+07,landfill_area_sq_meters,NaN,1.446208e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,King George Landfill Inc.,Sanitary Landfills,POINT(-77.30596 38.26964)
4,1835960684,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,104839.594100,0.028723,NaN,5.663107e+07,landfill_area_sq_meters,NaN,3.650000e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Jiangcungou Landfill,Sanitary Landfill,POINT(109.100563 34.245692)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4740,1835965296,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.341661,0.017325,NaN,3.870912e+04,landfill_area_sq_meters,NaN,1.693604e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Udong, Odongk, Kampong Speu, 855, Cambodia",Dumpsites,POINT(104.71550431136072 11.808565128092692)
4741,1835965297,PHL,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.301214,0.017325,NaN,3.865576e+04,landfill_area_sq_meters,NaN,1.691269e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Bangkal, Baraka, Bulacan, Central Luzon, 3013,...",Dumpsites,POINT(121.08773982254641 14.916148899405837)
4742,1835965298,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.380501,0.017325,NaN,3.744110e+04,landfill_area_sq_meters,NaN,1.638126e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Doun Kaev, Takeo, 885, Cambodia",Dumpsites,POINT(104.68932165937792 10.973150260323228)
4743,1835965299,NGA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.290846,0.012097,NaN,3.430273e+04,landfill_area_sq_meters,NaN,2.338728e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Tarkwa Bay, Amuwo Odofin, Lagos, 71510, Nigeria",Dumpsites,POINT(3.392517451014247 6.406222836614646)


In [4]:
# Define a function to split the name
def split_point(point):
    lon = float(point.split(' ')[0].split('(')[-1])
    lat = float(point.split(' ')[1].split(')')[0])
    return pd.Series((lat, lon))

In [5]:
latlons_TRACE[['lat', 'lon']] = latlons_TRACE['st_astext'].apply(split_point)

In [6]:
# SRON geolocation is only reported to two decimal points
# ie, longitude is 32.35. How far is that on earth's surface?
# Note that a degree of longitude is bigger at the equater than at poles

# Define the radius of the Earth (in kilometers)
R = 6371

# Set longitude difference
delta_longitude_degrees = 10.25 - 10.24

# Convert the change in longitude to radians
delta_longitude_radians = math.radians(delta_longitude_degrees)

# Calculate the distance
distance = R * delta_longitude_radians

print(f"Precision of SRON points: {distance} kilometers")

Precision of SRON points: 1.1119492664455637 kilometers


In [7]:
# Ok so our SRON points are accurate to like 1-2km

In [8]:
latlons_TRACE

,asset_id,iso3_country,original_inventory_sector,start_time,end_time,temporal_granularity,gas,emissions_quantity,emissions_factor,emissions_factor_units,...,capacity_factor,activity,activity_units,created_date,modified_date,asset_name,asset_type,st_astext,lat,lon
0,1835960681,KOR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,228071.908800,0.034650,NaN,...,NaN,6.582168e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Sudokwon,Sanitary Landfills,POINT(126.613998413085 37.5772357911111),37.577236,126.613998
1,1835960682,TUR,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131856.520200,0.030742,NaN,...,NaN,4.289130e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Odayeri,Sanitary Landfills,POINT(28.8581657 41.2161766),41.216177,28.858166
2,1835960683,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,131104.987200,0.034650,NaN,...,NaN,3.783697e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Laogang Landfill,Sanitary Landfill,POINT(121.874575 31.056294),31.056294,121.874575
3,1835960821,USA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,9868.230000,0.006824,NaN,...,NaN,1.446208e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,King George Landfill Inc.,Sanitary Landfills,POINT(-77.30596 38.26964),38.269640,-77.305960
4,1835960684,CHN,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,104839.594100,0.028723,NaN,...,NaN,3.650000e+06,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,Jiangcungou Landfill,Sanitary Landfill,POINT(109.100563 34.245692),34.245692,109.100563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4740,1835965296,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.341661,0.017325,NaN,...,NaN,1.693604e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Udong, Odongk, Kampong Speu, 855, Cambodia",Dumpsites,POINT(104.71550431136072 11.808565128092692),11.808565,104.715504
4741,1835965297,PHL,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,29.301214,0.017325,NaN,...,NaN,1.691269e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Bangkal, Baraka, Bulacan, Central Luzon, 3013,...",Dumpsites,POINT(121.08773982254641 14.916148899405837),14.916149,121.087740
4742,1835965298,KHM,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.380501,0.017325,NaN,...,NaN,1.638126e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Doun Kaev, Takeo, 885, Cambodia",Dumpsites,POINT(104.68932165937792 10.973150260323228),10.973150,104.689322
4743,1835965299,NGA,solid-waste-disposal,2021-01-01 00:00:00,2021-12-31 00:00:00,annual,ch4,28.290846,0.012097,NaN,...,NaN,2.338728e+03,estimated_tonnes_waste_in_place,2023-01-24 00:00:00,NaN,"Tarkwa Bay, Amuwo Odofin, Lagos, 71510, Nigeria",Dumpsites,POINT(3.392517451014247 6.406222836614646),6.406223,3.392517


In [16]:
# Sort by landfill size, so smaller facilities will be dropped if they are near larger ones
TRACE_sorted = latlons_TRACE.sort_values('activity', ascending=False).reset_index(drop=True)

In [17]:
# Drop points within 5km of another point. 

# Convert Lat/Lon to radians for use in haversine formula
TRACE_rad = np.deg2rad(TRACE_sorted[['lat', 'lon']])

# Construct BallTree
TRACE_tree = BallTree(TRACE_rad, metric='haversine')

# Initialize array to keep track of whether row should be kept
keep = np.ones(TRACE_sorted.shape[0], dtype=bool)

# Convert min_distance (km) to radians
# Earth's radius is approximately 6371 km
min_distance = 5  # distance in km
min_distance_rad = min_distance / 6371

# Iterate over each point
for i, row in enumerate(TRACE_rad.values):
    if keep[i]:  # if this point hasn't been marked for removal
        # Find all points within min_distance of this point
        inds = TRACE_tree.query_radius([row], r=min_distance_rad)[0]
        
        # Mark all *other* points as not to keep
        inds = inds[inds != i]  # remove self from list
        keep[inds] = False

# Filter DataFrame
print(TRACE_sorted.shape)
TRACE = TRACE_sorted[keep].reset_index(drop=True)
print(TRACE.shape)

(4745, 22)
(4320, 22)


In [11]:
latlons_SRON

,date,time_UTC,lat,lon,source_rate_t/h,uncertainty_t/h,uncertainty
0,20230304.0,18:43:18,32.20,-93.35,12,6.0,NaN
1,20230304.0,20:23:30,31.40,-103.55,12,6.0,NaN
2,20230304.0,20:23:24,31.23,-101.65,13,8.0,NaN
3,20230304.0,20:23:33,31.66,-104.02,23,10.0,NaN
4,20230305.0,08:10:15,17.93,78.45,12,5.0,NaN
...,...,...,...,...,...,...,...
770,20230304.0,08:29:41,19.26,72.87,54,23.0,NaN
771,20230304.0,08:30:31,22.09,72.83,5,4.0,NaN
772,20230304.0,08:32:45,30.17,71.42,20,12.0,NaN
773,20230304.0,08:33:02,31.46,73.07,22,11.0,NaN


In [15]:
# Sort by plume size
SRON_sorted = latlons_SRON.sort_values('source_rate_t/h', ascending=False).reset_index(drop=True)

In [13]:
# Drop points within 5km of another point. We don't actually have to do this.

# # Convert Lat/Lon to radians for use in haversine formula
# SRON_rad = np.deg2rad(SRON_sorted[['lat', 'lon']])

# # Construct BallTree
# SRON_tree = BallTree(SRON_rad, metric='haversine')

# # Initialize array to keep track of whether row should be kept
# keep = np.ones(SRON_sorted.shape[0], dtype=bool)

# # Convert min_distance (km) to radians
# # Earth's radius is approximately 6371 km
# min_distance = 10  # distance in km
# min_distance_rad = min_distance / 6371

# # Iterate over each point
# for i, row in enumerate(SRON_rad.values):
#     if keep[i]:  # if this point hasn't been marked for removal
#         # Find all points within min_distance of this point
#         inds = SRON_tree.query_radius([row], r=min_distance_rad)[0]
        
#         # Mark all *other* points as not to keep
#         inds = inds[inds != i]  # remove self from list
#         keep[inds] = False

# # Filter DataFrame
# print(SRON_sorted.shape)
# SRON = SRON_sorted[keep].reset_index(drop=True)
# print(SRON.shape)

(775, 7)
(524, 7)


In [30]:
# This code drops points within 5km of one another. It tries to minimize the number of points dropped.
# To do so, it calculates how many points are within the threshold of each point, and then drops
# points that have the most points nearby. This preserves the maximum number of points.
# However, it is VERY slow. Maybe use once for final version. 

# from sklearn.neighbors import BallTree
# import numpy as np
# import pandas as pd

# # Convert Lat/Lon to radians for use in haversine formula
# df_rad = np.deg2rad(latlons_TRACE[['lat', 'lon']])

# # Construct BallTree
# tree = BallTree(df_rad, metric='haversine')

# # Convert min_distance (km) to radians
# # Earth's radius is approximately 6371 km
# min_distance = 5  # distance in km
# min_distance_rad = min_distance / 6371

# # Compute the indices of points within min_distance for each point
# indices = tree.query_radius(df_rad, r=min_distance_rad)

# # Initialize keep array
# keep = np.ones(df_rad.shape[0], dtype=bool)

# # Repeat until all points are far enough away
# while True:
#     # Compute scores based on the number of close points
#     scores = np.array([len(inds) for inds in indices])

#     # If no point has more than one close point, we are done
#     if np.all(scores <= 1):
#         break

#     # Find the point with the highest score
#     i = np.argmax(scores)

#     # Mark this point as not to keep
#     keep[i] = False

#     # Remove this point from the list of close points for all other points
#     indices = [inds[inds != i] for inds in indices]

# # Filter DataFrame
# df_filtered = latlons_TRACE[keep]

In [18]:
SRON = latlons_SRON

In [19]:
# Now find points in both datasets that are within 10 km to try to match

# We have to remake these because we dropped points
# Convert Lat/Lon to radians for use in haversine formula
SRON_rad = np.deg2rad(SRON[['lat', 'lon']])
TRACE_rad = np.deg2rad(TRACE[['lat', 'lon']])

# Construct BallTree
TRACE_tree = BallTree(TRACE_rad, metric='haversine')

# Query the tree for TRACE points within 10 km
indices, distances = TRACE_tree.query_radius(SRON_rad, r=10/6371, return_distance=True)

matches = []
for i, (distance, index) in enumerate(zip(distances, indices)):
    for d, idx in zip(distance, index):
        matches.append({
            'lat_SRON': SRON.at[i, 'lat'],
            'lon_SRON': SRON.at[i, 'lon'],
            'idx_SRON': i,
            'lat_TRACE': TRACE.at[idx, 'lat'],
            'lon_TRACE': TRACE.at[idx, 'lon'],
            'idx_TRACE': idx,
            'distance': d * 6371  # Convert radians to km
        })

matches_df = pd.DataFrame(matches)

In [20]:
matches_df.shape

(213, 7)

In [21]:
matches_df

,lat_SRON,lon_SRON,idx_SRON,lat_TRACE,lon_TRACE,idx_TRACE,distance
0,28.70,77.31,15,28.686594,77.251543,1494,5.893611
1,28.70,77.31,15,28.716385,77.331861,1842,2.804513
2,28.70,77.31,15,28.624698,77.327405,118,8.543673
3,16.82,95.99,17,16.897999,96.021095,457,9.282890
4,17.09,96.12,19,17.114760,96.043403,3217,8.593504
...,...,...,...,...,...,...,...
208,23.76,86.46,763,23.726851,86.419195,3236,5.553002
209,50.22,18.64,764,50.263339,18.666973,1262,5.186790
210,13.83,99.94,768,13.800655,99.938980,3820,3.264862
211,19.26,72.87,770,19.286949,72.793920,1210,8.529313


In [22]:
# Drop 
matches_df.drop_duplicates(subset=['idx_TRACE'], inplace=True, ignore_index=True)

In [ ]:
matches_df

In [23]:
matches_df.to_csv('m.csv', index=False)